# Migration and sync of assets between prod and staging

## Summary

It will copy from one env to the other, will keep the match between IDs and we could potentially copy it back to prod from staging if we needed to

First thing we'd need to do is make a copy. Then there are new datasets being added right now- is the best process for them to add it to Prod and copy to stg, or to upload it into stg

Vizz would pass the script to WRI, we'd run it everytime we wanted to change anything over.
WRI preference is to create in production, then copy into staging.
If there are any changes needed to the datasets once they are in stg, the dataset will be updated in production, and then the script will copy the updates into staging.
Vizz will be taking regular backups as well here.

## Instructions

## Functions

This are the functions we need to create and sync assets from staging to production

In [7]:
import getpass
import requests as re
import json
from datetime import datetime

In [3]:
staging_server = "https://staging-api.globalforestwatch.org"
prod_server = "https://api.resourcewatch.org"

In [15]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [14]:
def auth(env='prod'):
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    print(f'You are login into {bcolors.HEADER}{bcolors.BOLD}{env}{bcolors.ENDC}')
    with re.Session() as s:
        headers = {'Content-Type': 'application/json'}
        payload = json.dumps({ 'email': f'{input(f"{bcolors.BOLD}Email: {bcolors.ENDC}")}',
                               'password': f'{getpass.getpass(prompt="Password: ")}'})
        response = s.post(f'{serverUrl[env]}/auth/login',  headers = headers,  data = payload)
        response.raise_for_status()
    return response.json().get('data').get('token')

In [ ]:
token = {
    'staging':
    'production':
}

In [ ]:
# @TODO migrate one day the payloads to data model classes and refactor to classes following inheritance
#from typing import List
#from pydantic import BaseModel, parse_obj_as
# class DatasetModel(BaseModel):

# class LayerModel(BaseModel):

# class widgetModel(BaseModel):

# class metadataModel(BaseModel):
     
# class vocabularyModel(BaseModel):


In [29]:
def getAssets(url, payload):
    response = re.get(url, payload)
    response.raise_for_status()
    return response.json()

def deleteAssets(url):
    response = re.delete(url, headers = {'Authorization':f'Bearer {token}'})
    response.raise_for_status()
    return response.status_code

def postAssets(url, body):
    response = re.post(url, data=json.dumps(body), headers = {'Authorization':f'Bearer {token}', 'Content-Type': 'application/json'})
    if response.status_code !=200:
        print(response.text)
        print(url)
        print(json.dumps(body))
    response.raise_for_status()
    return response.json()

def updateAssets(url, body):
    response = re.patch(url, data=json.dumps(body), headers = {'Authorization':f'Bearer {token}', 'Content-Type': 'application/json'})
    if response.status_code !=200:
        print(response.text)
        print(url)
        print(json.dumps(body))
    response.raise_for_status()
    return response.json()


def recreateDataset(dataset, toEnv = 'prod', destinationDatasetId = None):
    
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    if dataset.get('type')!='dataset':
        return None
    
    url = f'{serverUrl[env]}/v1/dataset/{destinationDatasetId}'
    body = {'dataset':{
        'application': dataset['attributes'].get('application'),
        'name': dataset['attributes'].get('name'),
        'connectorType': dataset['attributes'].get('connectorType'),
        'provider': dataset['attributes'].get('provider'),
        'published': dataset['attributes'].get('published'),
        'overwrite': dataset['attributes'].get('overwrite'),
        'mainDateField': dataset['attributes'].get('mainDateField'),
        'env': dataset['attributes'].get('env'),
        'geoInfo': dataset['attributes'].get('geoInfo'),
        'protected': dataset['attributes'].get('protected'),
        'legend': dataset['attributes'].get('legend'),
        'widgetRelevantProps': dataset['attributes'].get('widgetRelevantProps'),
        'layerRelevantProps': dataset['attributes'].get('layerRelevantProps')
        }
    }
    if dataset['attributes'].get('provider') == 'cartodb':
            body['dataset']['connectorUrl'] =  dataset['attributes'].get('connectorUrl')
    if dataset['attributes'].get('provider') == 'gee':
            body['dataset']['tableName'] =  dataset['attributes'].get('tableName')
    
    if destinationDatasetId:
        return updateAssets(url, body)
    else:
        return postAssets(url, body)
    
    

def recreateLayer(datasetId, layer, toEnv = 'prod', destinationLayerId = None):
    
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    if layer.get('type')!='layer':
        return None
    
    url = f'{serverUrl[env]}/v1/dataset/{datasetId}/layer/{destinationLayerId}'

    body = {
        'application': layer['attributes'].get('application'),
        'name': layer['attributes'].get('name'),
        'iso': layer['attributes'].get('iso'),
        'provider': layer['attributes'].get('provider'),
        'default': layer['attributes'].get('default'),
        'protected': layer['attributes'].get('protected'),
        'published': layer['attributes'].get('published'),
        'env': layer['attributes'].get('env'),
        'layerConfig': layer['attributes'].get('layerConfig'),
        'legendConfig': layer['attributes'].get('legendConfig'),
        'interactionConfig': layer['attributes'].get('interactionConfig'),
        'applicationConfig': layer['attributes'].get('applicationConfig'),
        'staticImageConfig': layer['attributes'].get('staticImageConfig')
    }
    
    if destinationLayerId:
        return updateAssets(url, body)
    else:
        return postAssets(url, body)

def recreateWidget(datasetId, widget, toEnv = 'prod', destinationWidgetId = None):
    
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    
    if widget.get('type')!='widget':
        return None
    
    url = f'{serverUrl[env]}/v1/dataset/{datasetId}/widget/{destinationWidgetId}'
    body = {
        'application': widget['attributes'].get('application'),
        'name': widget['attributes'].get('name'),
        'description': widget['attributes'].get('description'),
        'verified': widget['attributes'].get('verified'),
        'default': widget['attributes'].get('default'),
        'protected': widget['attributes'].get('protected'),
        'defaultEditableWidget': widget['attributes'].get('defaultEditableWidget'),
        'published': widget['attributes'].get('published'),
        'freeze': widget['attributes'].get('freeze'),
        'env': widget['attributes'].get('env'),
        'queryUrl': widget['attributes'].get('queryUrl'),
        'widgetConfig': widget['attributes'].get('widgetConfig'),
        'template': widget['attributes'].get('template'),
        'layerId': widget['attributes'].get('layerId')
    }
    if destinationWidgetId:
        return updateAssets(url, body)
    else:
        return postAssets(url, body)

def recreateMetadata(datasetId, metadata, layerId=None, widgetId=None, toEnv = 'prod'):
    
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    
    if metadata.get('type')!='metadata':
        return None
    if layerId and widgetId:
        raise Exception("layerId and widgetId not allowed at the same time")
    elif layerId:
        url = f'{serverUrl[env]}/v1/dataset/{datasetId}/layer/{layerId}/metadata'
    elif widgetId:
        url = f'{serverUrl[env]}/v1/dataset/{datasetId}/widget/{widgetId}/metadata'
    else:
        url = f'{serverUrl[env]}/v1/dataset/{datasetId}/metadata'
    
    body = {
        'application': metadata['attributes'].get('application'),
        'language': metadata['attributes'].get('language'),
        'description': metadata['attributes'].get('description'),
        'info': metadata['attributes'].get('info'),
    }
    if metadata['attributes'].get('name'):
        body['name'] = metadata['attributes'].get('name')
    
    try:
        response = postAssets(url, body)
    except Exception as e:
        response = updateAssets(url, body)
        pass
    
    return response

def recreateVocabulary(datasetId, vocabulary, toEnv = 'prod'):
    
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    
    if vocabulary.get('type')!='vocabulary':
        return None
    url = f"{serverUrl[env]}/v1/dataset/{datasetId}/vocabulary/{vocabulary['attributes']['name']}"
    body = {
        'application': vocabulary['attributes'].get('application'),
        'tags': vocabulary['attributes'].get('tags')
    }
    
    response = postAssets(url, body)
    return response

def getAssetList(Env = 'prod', datasetList=None):
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    url = f'{serverUrl[env]}/v1/dataset'
    payload={
        'application':'rw',
        'status':'saved',
        'published':'true',
        'includes':'widget,layer,vocabulary,metadata',
        'page[size]':1613982331640
    }
    if datasetList:
        payload['ids'] = datasetList
        return postAssets(url,payload)
    else:
        return getAssets(url,payload)
    
def backupAssets(Env = 'prod'):
    '''
    save a backup of production data just in case we need to recreate it again
    '''
    
    serverUrl = {
        'prod': staging_server,
        'staging': prod_server
    }
    url = f'{serverUrl[env]}/v1/dataset'
    payload = {
        'application':'rw',
        'status':'saved',
        'published':'true',
        'includes':'widget,layer,vocabulary,metadata',
        'page[size]':1613982331640
    }

    data = getAssets(url,payload)

    with open(f'RW_{Env}_backup_{datetime.now().strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
        json.dump(data, outfile)

def deleteDataFrom(env='staging'):
    serverUrl = {
        #'prod': staging_server,
        'staging': prod_server
    }
    userConfirmation = input(f'{bcolors.WARNING}Are you sure you want to delete everything in {env}:{bcolors.ENDC} Y/n')
    if userConfirmation == 'Y':
        for dataset in prodData['data']:
            try:
                status = deleteAssets(f"{staging_server}/v1/dataset/{dataset['id']}")
            except re.exceptions.HTTPError as err:
                pass

def copyAssets(datasetList, fromEnv='prod', toEnv='staging'):
    dataAssets = getAssetList(Env = fromEnv, datasetList = datasetList)
    if fromEnv == toEnv:
        raise NameError(f'fromEnv:{fromEnv} and toEnv:{toEnv} cannot be the same')
    try:
        resources=[]
        # @TODO:
        # Improve loop performance
        for dataset in dataAssets['data']:

            newDataset = recreateDataset(dataset)

            resources.append({
                'type': 'dataset',
                f'{fromEnv}Id':dataset.get('id'),
                f'{toEnv}Id': newDataset['data'].get('id')
            })

            for vocabulary in dataset['attributes'].get('vocabulary'):
                newVocabulary = recreateVocabulary(newDataset['data'].get('id'), vocabulary)
                resources.append({
                'type': 'vocabulary',
                f'{fromEnv}Id':vocabulary.get('id'),
                f'{toEnv}Id': newVocabulary['data']
            })

            for layer in dataset['attributes'].get('layer'):
                newLayer = recreateLayer(newDataset['data'].get('id'), layer)
                resources.append({
                'type': 'layer',
                f'{fromEnv}Id':layer.get('id'),
                f'{toEnv}Id': newLayer['data'].get('id')
            })

            for widget in dataset['attributes'].get('widget'):
                newWidget = recreateWidget(newDataset['data'].get('id'), widget)
                resources.append({
                'type': 'widget',
                f'{fromEnv}Id':widget.get('id'),
                f'{toEnv}Id': newWidget['data'].get('id')
            })

            for metadata in dataset['attributes'].get('metadata'):
                newMetadata = recreateMetadata(newDataset['data'].get('id'), metadata)
                resources.append({
                'type': 'metadata',
                f'{fromEnv}Id':metadata.get('id'),
                f'{toEnv}Id': newMetadata['data']
            })
    except:
        pass

    with open(f'RW_prod_staging_match_{datetime.now().strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
        json.dump(resources, outfile)
        
def syncAssets(syncList, fromEnv='prod', toEnv='staging'):
    return 0

## Get list of assets that we want to modify or sync

#### List of assets:

* we need to make sure that this list is in sync with the document we have shared with the assets

In [11]:
# in the future we can automate this listing base on the doc using the google sheet api
datasetsProd = []
layersProd = []
widgetsProd = []

### Backup Data in both environments

In [13]:
backupAssets('prod')
backupAssets('staging')

NameError: name 'backupAssets' is not defined

### Only do this if you want to clean data in staging. 
*You will need to be logged in

In [ ]:
#deleteDataFrom()

### Copy resources from production to staging. Depending on the asset size this operation might take time

In [23]:
copyAssets(datasetList)

SyntaxError: invalid syntax (<ipython-input-23-4f56ea9b8123>, line 1)

### Open sync list of assets match it with list and update it.